## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

--2022-12-01 08:41:42--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Risoluzione di www.dropbox.com (www.dropbox.com)... 162.125.69.18, 2620:100:6025:18::a27d:4512
Connessione a www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 302 Found
Posizione: /s/raw/xvjzaxzz3ysphme/data_000637.txt [segue]
--2022-12-01 08:41:44--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Riutilizzo della connessione esistente a www.dropbox.com:443.
Richiesta HTTP inviata, in attesa di risposta... 302 Found
Posizione: https://ucc1c319da73cba4feeaf9fa0674.dl.dropboxusercontent.com/cd/0/inline/BxwnB-Ixq11cihyM8mgRWn4pi9-wa6jTvSiNP6falG1tDlUdtSJUbo5f4-td_p3mnUoKbPE3tlXZTsif_ZUgeVLDsbkdbzkuXICVE25LnKLAyB9g-0tvAAcea2_fUpODX_lBcMhzfvW6jJpZJvcZeaJ_lkdr7b0-084cI4-zHHk3og/file# [segue]
--2022-12-01 08:41:45--  https://ucc1c319da73cba4feeaf9fa0674.dl.dropboxusercontent.com/cd/0/inline/BxwnB-Ixq11cihyM8mgRWn4pi9-wa6jTvSiNP6falG

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [20]:
#1

import numpy as np
import pandas as pd

file_name = "~/data/data_000637.txt"
data = pd.read_csv(file_name, header=0, nrows=12000)
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
11995,1,0,139,3869200286,3251,0
11996,1,0,62,3869200286,3246,4
11997,1,0,58,3869200286,3246,11
11998,1,0,61,3869200286,3251,17


In [22]:
#2

diff = data['ORBIT_CNT'].diff()
diff1 = diff[diff>0]
diff1.index

0        NaN
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
11995    0.0
11996    0.0
11997    0.0
11998    0.0
11999    0.0
Name: ORBIT_CNT, Length: 12000, dtype: float64


Int64Index([   43,   128,   255,   353,   462,   551,   639,   767,   895,
              946,
            ...
            10992, 11135, 11190, 11269, 11378, 11519, 11576, 11648, 11797,
            11877],
           dtype='int64', length=124)

In [18]:
diff2 = data.filter(items = diff1.index-1, axis=0)
diff2

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
42,1,0,57,3869200167,3187,17
127,1,0,62,3869200168,3538,4
254,1,0,49,3869200169,2766,1
352,1,1,3,3869200170,3377,11
461,1,0,64,3869200171,3538,7
...,...,...,...,...,...,...
11518,1,0,60,3869200281,2642,25
11575,1,0,139,3869200282,3255,0
11647,1,0,82,3869200283,3356,27
11796,1,1,22,3869200284,3524,23


In [23]:
x = diff2.mean(axis=0)['BX_COUNTER']

In [24]:
print(x)

3205.201612903226


In [37]:
#3, 4

data_tot = pd.read_csv(file_name, header=0)
times = data_tot['TDC_MEAS']*25/30+data_tot['BX_COUNTER']*25
print(times)
times = np.sum(times)
print("Total time:", times, "ns")

data['times'] = data['TDC_MEAS']*25/30+data['BX_COUNTER']*25
data


0          59371.666667
1          59372.500000
2          63848.333333
3          63965.833333
4          69020.833333
               ...     
1310715    19061.666667
1310716    19084.166667
1310717    19100.000000
1310718    19225.000000
1310719    19065.000000
Length: 1310720, dtype: float64
Total time: 58377517177.50002 ns


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,times
0,1,0,123,3869200167,2374,26,59371.666667
1,1,0,124,3869200167,2374,27,59372.500000
2,1,0,63,3869200167,2553,28,63848.333333
3,1,0,64,3869200167,2558,19,63965.833333
4,1,0,64,3869200167,2760,25,69020.833333
...,...,...,...,...,...,...,...
11995,1,0,139,3869200286,3251,0,81275.000000
11996,1,0,62,3869200286,3246,4,81153.333333
11997,1,0,58,3869200286,3246,11,81159.166667
11998,1,0,61,3869200286,3251,17,81289.166667


In [40]:
#5

data = data[data['HEAD']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices